In [ ]:
import os
import scanpy as sc
import pandas as pd
import numpy as np

print("Done.")

In [1]:
headdir="/oak/stanford/groups/smontgom/amarder/t21-proj"
disease_status="Healthy"
sampletype="Liver"
suffix="subset"
suffixDirec=""
if suffix=="subset":
    suffixDirec="subset"
else:
    suffixDirec="full"
os.system("mkdir -p "+headdir + "/out/" + suffixDirec)

def return_fileName(headdir,disease_status,sampletype,suffix):
    foutpath=""
    if suffix=="subset":
        fout="10X_" + disease_status + "_" + sampletype + ".umap.subset.cells_removed.h5ad"
        foutpath=headdir + "/out/subset/data/" + fout
    # else:
    #     fout="10X_" + disease_status + "_" + sampletype + ".h5ad"
    #     foutpath="/oak/stanford/groups/smontgom/amarder/data/t21/ScanpyObjects" + "/" + fout
    return foutpath

foutpath=return_fileName(headdir,"Healthy",sampletype,suffix)
print("\n * Reading in data..." + foutpath)
adata=sc.read_h5ad(foutpath)

foutpath=return_fileName(headdir,"DownSyndrome",sampletype,suffix)
print("\n * Reading in data..." + foutpath)
adata2=sc.read_h5ad(foutpath)

# disease_status="DownSyndrome"

print("Data read completed.")

Done.

 * Reading in data.../oak/stanford/groups/smontgom/amarder/t21-proj/out/subset/data/10X_Healthy_Liver.umap.subset.cells_removed.h5ad

 * Reading in data.../oak/stanford/groups/smontgom/amarder/t21-proj/out/subset/data/10X_DownSyndrome_Liver.umap.subset.cells_removed.h5ad
Data read completed.


In [13]:
# pd.set_option('display.max_columns', None)
# print(adata3.obs)
# pd.set_option('display.max_columns', 20)


                          n_genes  patient       sample  n_genes_by_counts  \
AAACCCAAGGAACGCT-1-0-0       4870    15633      L15633A               4870   
AAACCCAAGTAAACAC-1-0-0       1167    15633      L15633A               1167   
AAACCCAAGTTACTCG-1-0-0       1047    15633      L15633A               1047   
AAACCCACAAGTTTGC-1-0-0       3707    15633      L15633A               3707   
AAACCCACACAGAGCA-1-0-0       2024    15633      L15633A               2024   
...                           ...      ...          ...                ...   
CAAGCTAGTCCATAGT-1-0-0-1     4582    15532  T21 15532 A               4582   
CAAGCTATCTGAGTCA-1-0-0-1     6004    15532  T21 15532 A               6004   
CAAGGGAAGCTGGCTC-1-0-0-1     4305    15532  T21 15532 A               4305   
CAAGGGAAGGATGGCT-1-0-0-1     3842    15532  T21 15532 A               3842   
CAAGGGACAGGTCAAG-1-0-0-1     4071    15532  T21 15532 A               4071   

                          total_counts  total_counts_mt  pct_co

In [102]:
a=[columnName[8:] for columnName in adata.obs.columns if 'leiden_v' in columnName]
colName1="leiden_v" + str(max([int(x) for x in a if x.isdigit()]))
print("\n * Cell type column name to use: " + colName1)
adata.obs["leiden_names"]=adata.obs[colName1]

a=[columnName[8:] for columnName in adata2.obs.columns if 'leiden_v' in columnName]
colName2="leiden_v" + str(max([int(x) for x in a if x.isdigit()]))
print("\n * Cell type column name to use: " + colName2)
adata2.obs["leiden_names"]=adata2.obs[colName2]

adata=adata[:1000,]
adata2=adata2[:1000,]
adata3=adata.concatenate(adata2)

Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.



 * Cell type column name to use: leiden_v7

 * Cell type column name to use: leiden_v10


In [115]:
# healthy_cells = np.unique(adata.obs[colName1])
# ds_cells = np.unique(adata2.obs[colName2])
# clusters_for_DE = [cell_type for cell_type in healthy_cells if cell_type in ds_cells]

print(clusters_for_DE)

['B cells', 'Erythroid', 'HSC/Progenitors', 'Mast cells', 'Megakaryocytes', 'Myeloid', 'NK/T cells', 'Stroma']


In [168]:
healthy_cells = np.unique(adata.obs["cell_type_groups"])
ds_cells = np.unique(adata2.obs["cell_type_groups"])
clusters_for_DE = [cell_type for cell_type in healthy_cells if cell_type in ds_cells]
subDirec="DE_cell_type_groups"
os.system("mkdir -p "+headdir + "/out/" + suffixDirec +"/" + subDirec)

# cell_type=clusters_for_DE[0]
for cell_type in clusters_for_DE:
    
    ind=np.where(adata3.obs[["cell_type_groups"]]==cell_type)[0]
    adata4=adata3[ind,] #in cell_types_of_interest
    sc.tl.rank_genes_groups(adata4, 'environment', method='wilcoxon')

    cell_type_filename = cell_type.replace("/","_")
    filename_out=headdir + "/out/" + suffixDirec +"/" + subDirec + "/" + cell_type_filename + ".txt"

    sc.get.rank_genes_groups_df(adata4,group='Down Syndrome').to_csv(filename_out,na_rep='NA',index=False)

##########################################
# # random sampling:
# adata4.obs["randNumCol"] = np.random.permutation(adata4.obs["environment"].values)

# # random 50/50 labels
# import random
# adata4.obs["randNumCol"] = random.choices(["Down Syndrome","Healthy"],k=adata4.shape[0])

# sc.tl.rank_genes_groups(adata4, 'randNumCol', method='wilcoxon')
# sc.get.rank_genes_groups_df(adata4,group="Down Syndrome")


/home/amarder/anaconda3/envs/minimal_env/lib/python3.6/site-packages/anndata/_core/anndata.py:1229: ImplicitModificationWarning: Initializing view as actual.
  "Initializing view as actual.", ImplicitModificationWarning
Trying to set attribute `.obs` of view, copying.
... storing 'sample' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'sorting' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'environment' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'sangerID' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'patient_sample' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v1' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v2' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v3' as ca

... storing 'leiden_v6' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v7_split' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v7' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'annotations_atac' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'annotations_atac_v2' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'numerical_labels' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_names' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v8' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v9' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v10_split' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v10' as categorical
/home/amarder/anaconda3/envs/minimal_env

Trying to set attribute `.obs` of view, copying.
... storing 'environment' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'sangerID' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'patient_sample' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v1' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v2' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v3' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v4' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v5' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v5_split' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v6' as categorical
Trying to set attribute

ValueError: Could not calculate statistics for groups Down Syndrome since they only contain one sample.

In [169]:
healthy_cells = np.unique(adata.obs[colName1])
ds_cells = np.unique(adata2.obs[colName2])
clusters_for_DE = [cell_type for cell_type in healthy_cells if cell_type in ds_cells]
subDirec="DE_leiden_names"
os.system("mkdir -p "+headdir + "/out/" + suffixDirec +"/" + subDirec)

# cell_type=clusters_for_DE[0]
for cell_type in clusters_for_DE:
    
    ind=np.where(adata3.obs[["leiden_names"]]==cell_type)[0]
    adata4=adata3[ind,] #in cell_types_of_interest
    sc.tl.rank_genes_groups(adata4, 'environment', method='wilcoxon')

    cell_type_filename = cell_type.replace("/","_")
    filename_out=headdir + "/out/" + suffixDirec +"/" + subDirec + "/" + cell_type_filename + ".txt"

    sc.get.rank_genes_groups_df(adata4,group='Down Syndrome').to_csv(filename_out,na_rep='NA',index=False)

##########################################
# # random sampling:
# adata4.obs["randNumCol"] = np.random.permutation(adata4.obs["environment"].values)

# # random 50/50 labels
# import random
# adata4.obs["randNumCol"] = random.choices(["Down Syndrome","Healthy"],k=adata4.shape[0])

# sc.tl.rank_genes_groups(adata4, 'randNumCol', method='wilcoxon')
# sc.get.rank_genes_groups_df(adata4,group="Down Syndrome")


/home/amarder/anaconda3/envs/minimal_env/lib/python3.6/site-packages/anndata/_core/anndata.py:1229: ImplicitModificationWarning: Initializing view as actual.
  "Initializing view as actual.", ImplicitModificationWarning
Trying to set attribute `.obs` of view, copying.
... storing 'sample' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'sorting' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'environment' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'sangerID' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'patient_sample' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v1' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v2' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v3' as ca

... storing 'leiden_v6' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v7_split' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v7' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'annotations_atac' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'annotations_atac_v2' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'numerical_labels' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_names' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v8' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v9' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v10_split' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v10' as categorical
/home/amarder/anaconda3/envs/minimal_env

Trying to set attribute `.obs` of view, copying.
... storing 'environment' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'sangerID' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'patient_sample' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v1' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v2' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v3' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v4' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v5' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v5_split' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v6' as categorical
Trying to set attribute

Trying to set attribute `.obs` of view, copying.
... storing 'numerical_labels' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_names' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v8' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v9' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v10_split' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'leiden_v10' as categorical
/home/amarder/anaconda3/envs/minimal_env/lib/python3.6/site-packages/anndata/_core/anndata.py:1229: ImplicitModificationWarning: Initializing view as actual.
  "Initializing view as actual.", ImplicitModificationWarning
Trying to set attribute `.obs` of view, copying.
... storing 'sample' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'sorting' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'e

In [ ]:
enr_res = gseapy.enrichr(gene_list=glist,
                     organism='Human',
                     gene_sets='GO_Biological_Process_2018',
                     description='pathway',
                     cutoff = 0.5)
